In [ ]:
from imutils.video import VideoStream
import numpy as np
from imutils.video import FPS
import imutils
import time
import cv2
from keras.models import load_model
import os

CLASSES = ['background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']
#CLASSES = ['motorbike', 'person']
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

# load our serialized model from disk
print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe('deploy.prototxt.txt', 'deploy.caffemodel')

print('Loading helmet model...')
loaded_model = load_model('helmet_model.h5')
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# initialize the video stream,
print("[INFO] starting video stream...")

# Loading the video file
cap = cv2.VideoCapture('/Users/utkarsh/Downloads/helmet.mp4') 

# Starting the FPS calculation
fps = FPS().start()

# loop over the frames from the video stream
while True:

    try:

        ret, frame = cap.read()

        frame = imutils.resize(frame, width=600, height=600)

        # grab the frame dimensions and convert it to a blob
        (h, w) = frame.shape[:2]
        
        # Resizing to a fixed 300x300 pixels and normalizing it.
        # Creating the blob from image to give input to the Caffe Model
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 0.007843, (300, 300), 127.5)

        # pass the blob through the network and obtain the detections and predictions
        net.setInput(blob)

        detections = net.forward()  # getting the detections from the network
        
        persons = []
        #person_roi = []
        motorbi = []
        
        # loop over the detections
        for i in np.arange(0, detections.shape[2]):
            # extract the confidence associated with the prediction
            confidence = detections[0, 0, i, 2]
            
            # filter out weak detections by ensuring the confidence
            # is greater than minimum confidence
            if confidence > 0.5:
                
                # extract index of class label from the detections
                idx = int(detections[0, 0, i, 1])
                
                if idx == 15:
                    box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                    (startX, startY, endX, endY) = box.astype("int")
                    persons.append((startX, startY, endX, endY))

                if idx == 14:
                    box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                    (startX, startY, endX, endY) = box.astype("int")
                    motorbi.append((startX, startY, endX, endY))  

        xsdiff = 0
        xediff = 0
        ysdiff = 0
        yediff = 0
        p = ()
        
        for i in motorbi:
            mi = float("Inf")
            for j in range(len(persons)):
                xsdiff = abs(i[0] - persons[j][0])
                xediff = abs(i[2] - persons[j][2])
                ysdiff = abs(i[1] - persons[j][1])
                yediff = abs(i[3] - persons[j][3])

                if (xsdiff+xediff+ysdiff+yediff) < mi:
                    mi = xsdiff+xediff+ysdiff+yediff
                    p = persons[j]

            if len(p) != 0:

            # display the prediction
                label = "{}".format(CLASSES[14])
                print("[INFO] {}".format(label))
                cv2.rectangle(frame, (i[0], i[1]), (i[2], i[3]), COLORS[14], 2)
                y = i[1] - 15 if i[1] - 15 > 15 else i[1] + 15
                cv2.putText(frame, label, (i[0], y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[14], 2)   
                label = "{}".format(CLASSES[15])
                print("[INFO] {}".format(label))

                cv2.rectangle(frame, (p[0], p[1]), (p[2], p[3]), COLORS[15], 2)
                y = p[1] - 15 if p[1] - 15 > 15 else p[1] + 15

                roi = frame[p[1]:p[1]+(p[3]-p[1])//4, p[0]:p[2]]
                print(roi)
                if len(roi) != 0:
                    img_array = cv2.resize(roi, (50,50))
                    gray_img = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
                    img = np.array(gray_img).reshape(1, 50, 50, 1)
                    img = img/255.0
                    prediction = loaded_model.predict_proba([img])
                    cv2.rectangle(frame, (p[0], p[1]), (p[0]+(p[2]-p[0]), p[1]+(p[3]-p[1])//4), COLORS[0], 2)
                    cv2.putText(frame, str(round(prediction[0][0],2)), (p[0], y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[0], 2)
                    print("prediction",prediction)
                    
                    # converting video to frame if prediciton < 50%
                    currentFrame = 0
                    for i in prediction:
                        if i < 0.5: 
                            #name = '/Users/utkarsh/Desktop/output_data/' + str(currentFrame) + '.jpg'
                            #print ('Creating...' + name)
                            #cv2.imwrite(name,frame)
                            cv2.imwrite('/Users/utkarsh/Desktop/output_data/'+ str(i) +'.jpg', frame)
                            cv2.imshow('frame', frame)
                            currentFrame += 1
                        
                    
    except:
        pass
        
        """currentFrame = 0
        if prediction < 0.5: 
            ret, frame = cap.read()
            name = '/Users/utkarsh/Desktop/output_data/' + str(currentFrame) + '.jpg'
            print ('Creating...' + name)
            currentFrame += 1
            cv2.imwrite(name,frame)
            #location = '/Users/utkarsh/Desktop/output_data/'
            #cv2.imwrite(os.path.join(location), frame) """
                
        
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'): # if 'q' key is pressed, exit
        break
     
    # update the FPS counter
    fps.update()

# stop the timer and display FPS information
fps.stop()

print("[INFO] elapsed time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
 
cv2.destroyAllWindows()
cap.release()   # Closing the video stream 

Using TensorFlow backend.


[INFO] loading model...
Loading helmet model...
[INFO] starting video stream...
[INFO] motorbike
[INFO] person
[[[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [ 95 135  66]
  ...
  [124 124 124]
  [123 123 123]
  [151 125  64]]

 ...

 [[151 125  64]
  [151 125  64]
  [ 95 135  66]
  ...
  [131 131 131]
  [131 131 131]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [ 95 135  66]
  ...
  [131 131 131]
  [130 130 130]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [ 95 135  66]
  ...
  [130 130 130]
  [129 129 129]
  [151 125  64]]]
prediction [[0.76486725]]
[INFO] motorbike
[INFO] person
[[[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
  [151 125  

[INFO] motorbike
[INFO] person
[[[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [146 149 151]
  ...
  [ 62  61  63]
  [ 61  60  63]
  [151 125  64]]

 ...

 [[151 125  64]
  [151 125  64]
  [143 145 147]
  ...
  [ 68  60  55]
  [ 68  60  55]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [141 144 149]
  ...
  [ 68  60  55]
  [ 68  60  55]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [143 146 151]
  ...
  [ 68  60  55]
  [ 68  60  55]
  [151 125  64]]]
prediction [[0.35351738]]
[INFO] motorbike
[INFO] person
[[[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [148 152 154]
  ...
  [145 144 146]
  [1

[INFO] motorbike
[INFO] person
[[[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [132 132 132]
  ...
  [140 140 140]
  [138 138 138]
  [151 125  64]]

 ...

 [[151 125  64]
  [151 125  64]
  [131 131 131]
  ...
  [ 95 135  66]
  [145 145 145]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [131 131 131]
  ...
  [ 95 135  66]
  [141 141 141]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [131 131 131]
  ...
  [ 95 135  66]
  [138 138 138]
  [151 125  64]]]
prediction [[0.74508613]]
[INFO] motorbike
[INFO] person
[[[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [132 132 132]
  ...
  [140 140 140]
  [1

[INFO] motorbike
[INFO] person
[[[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [ 95 135  66]
  ...
  [ 71  62  62]
  [ 71  63  65]
  [151 125  64]]

 ...

 [[151 125  64]
  [151 125  64]
  [134 134 134]
  ...
  [ 97  78  81]
  [ 94  77  85]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [127 127 127]
  ...
  [ 90  67  67]
  [ 97  76  82]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [110 110 110]
  ...
  [ 93  72  67]
  [135 114 113]
  [151 125  64]]]
prediction [[0.0964321]]
[INFO] motorbike
[INFO] person
[[[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [ 95 135  66]
  ...
  [ 71  62  62]
  [ 7

...

[INFO] motorbike
[INFO] person
[[[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [129 149 135]
  ...
  [130 129 131]
  [132 131 133]
  [151 125  64]]

 ...

 [[151 125  64]
  [151 125  64]
  [149 155 142]
  ...
  [145 144 146]
  [147 146 148]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [149 155 142]
  ...
  [145 144 146]
  [147 146 148]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [149 155 142]
  ...
  [145 144 146]
  [146 145 147]
  [151 125  64]]]
prediction [[0.90023446]]
[INFO] motorbike
[INFO] person
[[[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [129 149 135]
  ...
  [129 128 130]
  [1

[INFO] motorbike
[INFO] person
[[[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [ 20  14 215]
  ...
  [ 54  45  70]
  [ 54  46  70]
  [151 125  64]]

 ...

 [[151 125  64]
  [151 125  64]
  [ 17  16 216]
  ...
  [ 95 135  66]
  [ 95 135  66]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [ 16  16 216]
  ...
  [ 95 135  66]
  [ 95 135  66]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [ 15  16 216]
  ...
  [ 95 135  66]
  [ 95 135  66]
  [151 125  64]]]
prediction [[0.857345]]
[INFO] motorbike
[INFO] person
[[[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [ 95 135  66]
  ...
  [ 54  45  70]
  [ 54

[INFO] motorbike
[INFO] person
[[[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [138 135 136]
  ...
  [110 107 110]
  [109 107 110]
  [151 125  64]]

 ...

 [[151 125  64]
  [151 125  64]
  [139 139 139]
  ...
  [ 58  51  53]
  [ 64  56  58]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [138 138 138]
  ...
  [ 59  49  51]
  [ 66  55  57]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [137 137 137]
  ...
  [ 61  47  48]
  [ 67  52  53]
  [151 125  64]]]
prediction [[0.85967886]]
[INFO] motorbike
[INFO] person
[[[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [138 135 136]
  ...
  [110 107 110]
  [1

prediction [[0.66048586]]
[INFO] motorbike
[INFO] person
[[[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [161 160 162]
  ...
  [127 122 123]
  [128 123 124]
  [151 125  64]]

 ...

 [[151 125  64]
  [151 125  64]
  [165 165 165]
  ...
  [127 122 123]
  [127 122 123]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [165 165 165]
  ...
  [128 123 124]
  [128 123 124]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [165 165 165]
  ...
  [127 122 123]
  [127 122 123]
  [151 125  64]]]
prediction [[0.87394726]]
[INFO] motorbike
[INFO] person
[[[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [161 160 162]


prediction [[0.21240036]]
[INFO] motorbike
[INFO] person
[]
[INFO] motorbike
[INFO] person
[]
[INFO] motorbike
[INFO] person
[]
[INFO] motorbike
[INFO] person
[]
[INFO] motorbike
[INFO] person
[]
[INFO] motorbike
[INFO] person
[[[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [147 141 138]
  ...
  [ 54 204  48]
  [ 40 210  42]
  [151 125  64]]

 ...

 [[151 125  64]
  [151 125  64]
  [147 147 146]
  ...
  [ 53 204  45]
  [ 37 211  33]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [147 147 146]
  ...
  [ 53 202  47]
  [ 37 210  34]
  [151 125  64]]

 [[151 125  64]
  [151 125  64]
  [147 147 146]
  ...
  [ 53 202  47]
  [ 37 210  34]
  [151 125  64]]]
prediction [[0.92412126]]
[INFO] motorbike
[INFO] person
[[[151 125  64]
  [151 125  64]
  [151 125  64]
  ...
  [151 125  64]
  [151 125  64]
